# Chapter 10: Machine Learning with MLlib
Christoph Windheuser    
June, 2022   
Python examples of chapter 10 (page 285 ff) in the book *Learning Spark*

In [3]:
# Import required python spark libraries
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.streaming import StreamingContext


In [7]:
# create a SparkSession
# This requires access to the internet. If executed offline, an error is thrown

spark = (SparkSession \
         .builder \
         .appName("Chapter_8") \
         .getOrCreate())


NameError: name 'SparkSession' is not defined

In [5]:
# SparkSession 
spark = (
    SparkSession.builder
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
    .getOrCreate()
)


NameError: name 'SparkSession' is not defined